In [36]:
import pandas as pd
import calendar
import datetime
from datetime import timedelta, date

#导入数据，选出用来做日表用的几列
fpath = "/Users/apple/Desktop/Automated_report.xlsx"
df = pd.read_excel(fpath, '每日电商数据')
df = df.iloc[:, 0:8]

#先写个可以提取出某一天六个电商数据的function
def extract_data(date):
    data = df[df.黄石爱康医院 == date].copy()
    #给列重新命个名
    columns = ['时间','访客数','下单人数', '下单笔数', '支付人数', '支付订单数', '支付件数', '支付金额']
    #给行重新编码
    data.columns = columns
    data = data.reset_index(drop = True)
    return data

def get_report(y1, m1, d1, y2=None, m2=None, d2=None):
    """给一个日期或者一个日期范围，得出对应的报表。
    起始日期的年月日分别是y1, m1, d1; 
    结束日期的年月日分别是y2, m2, d2。
    如果想要某一天的日报，则只需输入起始日期的三个参数。
    """
    start = datetime.datetime(y1, m1, d1, 0, 0)
    data_list = [extract_data(start)]
    if y2 is not None:
        end = datetime.datetime(y2, m2, d2, 0, 0)
        while start != end:
            start += timedelta(days = 1)
            data_list.append(extract_data(start))
    if len(data_list) == 1:
        data = data_list[0]
    else:
        data = pd.concat(data_list).groupby(level = 0).sum()
        data['时间'] = "{}/{}/{} - {}/{}/{}".format(y1, m1, d1, y2, m2, d2)
    data['单位'] = ['黄石医院','岳阳医院','赤壁医院', '巢湖医院','黄石社区','爱康云']
    data['下单/访客人数'] = data.下单人数 / data.访客数
    data['支付/下单人数'] = data.支付人数 / data.下单人数
    order = ['时间', '单位', '访客数','下单人数','下单/访客人数',
             '下单笔数', '支付人数', '支付/下单人数', '支付订单数', '支付件数', '支付金额']
    data = data[order]
    #加入是数字的那几列的合计
    data.loc[7] = data.iloc[:, 2:11].apply(lambda x:x.sum())
    #修改一下最后一行
    data.at[7, '时间'] = data.loc[5]['时间']
    data.at[7, '单位'] = '合计'
    data.at[7, '下单/访客人数'] = data.loc[7]['下单人数'] / data.loc[7]['访客数']
    data.at[7, '支付/下单人数'] = data.loc[7]['支付人数'] / data.loc[7]['下单人数']
    return data



In [38]:
get_report(2021, 5, 14, 2021, 5, 16)

,时间,单位,访客数,下单人数,下单/访客人数,下单笔数,支付人数,支付/下单人数,支付订单数,支付件数,支付金额
0,2021/5/14 - 2021/5/16,黄石医院,205.0,60.0,0.292683,91.0,60.0,1.000000,89.0,182.0,114237.50
1,2021/5/14 - 2021/5/16,岳阳医院,17.0,5.0,0.294118,6.0,5.0,1.000000,6.0,6.0,738.00
2,2021/5/14 - 2021/5/16,赤壁医院,32.0,17.0,0.531250,57.0,17.0,1.000000,55.0,55.0,85610.00
3,2021/5/14 - 2021/5/16,巢湖医院,894.0,377.0,0.421700,418.0,374.0,0.992042,405.0,541.0,52888.25
4,2021/5/14 - 2021/5/16,黄石社区,39.0,13.0,0.333333,18.0,13.0,1.000000,16.0,27.0,15733.51
5,2021/5/14 - 2021/5/16,爱康云,133.0,8.0,0.060150,8.0,7.0,0.875000,7.0,7.0,1856.00
7,2021/5/14 - 2021/5/16,合计,1320.0,480.0,0.363636,598.0,476.0,0.991667,578.0,818.0,271063.26


In [ ]:
def add_ratio(df, year, month, day):
    """
    df: 已有的环比数据
    year, month, day: 今天日期

    更新今天的环比数据。
    """
    df = df.copy()
    
    #记录今天新有数据对应的日期（也就是昨天的日期）
    today = datetime(year, month, day) - timedelta(days = 1)

    #先确认today是星期几
    weekday = today.weekday()

    #得到这一周周一的日期
    monday = today - timedelta(days=weekday)

    #得到从那周周一，一直到today的数据
    data = get_report(monday.year, monday.month, monday.day, today.year,
                      today.month, today.day)

    #提取出我们需要的列数，把它转换为一个list（到这里我们得到了除时间以外所有需要的数据）
    data_list = data.iloc[-1, [2, 3, 4, 6, 7, 9, 10]].to_list()

    #把时间整好
    time = "{}周（{}.{}-{}.{}）".format(today.isocalendar()[1] + 1,
                                                      monday.month, monday.day,
                                       today.month, today.day)
    #把时间放到list的开头
    data_list.insert(0, time)

    #用num代表df的行数
    num = df.shape[0]

    #如果今天是周一，则给表格新加一行（如果不是周一，只要修改最新一周的那一行就行了）
    if today.weekday() == 0:
        #新加一行
        df.loc[num] = data_list

        #此时表格多了一行
        num += 1

        #把这一行放在应有的位置（倒数第三行）（我暂时没有找到更好的方法）
        total, dist, new_week = df.loc[num-3].copy(), df.loc[num-2].copy(),\
                              df.loc[num - 1].copy()

        df.loc[num-3], df.loc[num-2], df.loc[num - 1] = new_week, total, dist

    else:
        #如果不是周一，直接修改倒数第三行
        df.loc[num - 3] = data_list

    #修改合计
    df.loc[num-2] = df.iloc[:num-2, 1:8].apply(lambda x:x.sum())
    df.at[num - 2, '下单/访问'] = df.loc[num - 2]['下单人数'] / df.loc[num - 2]['访问有赞人数']
    df.at[num - 2, '支付/下单'] = df.loc[num - 2]['支付人数'] / df.loc[num - 2]['下单人数']
    df.at[num -2, '时间'] = '合计'

    #修改最后一行
    amount = str(40000000 - df.iloc[num - 2]["支付金额"])
    df.at[num - 1, '时间'] = "距离完成4000万GMV还差{}万{}元".format(amount[:4], amount[4:])

    return df